In [1]:
from isaac.constants import BASIC_TRAINING_COLS, YOKED_TRAINING_COLS
from rpy2.robjects import r, pandas2ri
import os
from tqdm import tqdm
pandas2ri.activate()

In [2]:
ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]

In [3]:
rdata_path = "data/exp1.rdata"
r['load'](rdata_path)

'sw','tw'


In [4]:
all_trials = r['ls.clips']

is_passive = (r['dfc.l'].condition == '0') & (r['dfc.l'].practice == 0.)  & (r['dfc.l'].exclude == 0)

responses = r['dfc.l'][["trueMass", "trueRelationship"]]
passive_responses = responses[is_passive]

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [5]:
PASSIVE_PATH = "data/passive_trials_exp1.h5"

if os.path.exists(PASSIVE_PATH):
    os.remove(PASSIVE_PATH)

In [6]:
flat_trials = []
trial_i = 0

for participant in all_trials:
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
        # print(trial["idControlledObject"].nunique() > 1, is_active[trial_i])
        flat_trials.append(trial)
        trial_i += 1

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [7]:
passive_trials = [trial for trial, is_trial_passive in zip(flat_trials, is_passive) if is_trial_passive]

In [8]:
def add_velocity_from_positions(trial):
    POSITION_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y"]]
    VELOCITY_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["vx", "vy"]]
    S_PER_FRAME = 1./60

    positions = trial[POSITION_COLS]
    shifted_positions = positions.shift(1).fillna(positions.iloc[0])
    trial[VELOCITY_COLS] = (shifted_positions - positions) / S_PER_FRAME

In [9]:
velocities_not_recorded = 0
count = 0
hdf_path = PASSIVE_PATH


for trial_i, trial in tqdm(enumerate(passive_trials), total=len(passive_trials)):

    response = passive_responses.iloc[trial_i]
    
    if not set(ALTERNATIVE_TRAINING_COLS).issubset(trial.columns):
        velocities_not_recorded += 1
        add_velocity_from_positions(trial)
        
    trial = trial.rename({alt_col: col for col, alt_col in 
                          zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")

    trial["C_none"] = (trial["idControlledObject"] == "none")
    trial["C_O1"] = (trial["idControlledObject"] == "object1")
    trial["C_O2"] = (trial["idControlledObject"] == "object2")
    trial["C_O3"] = (trial["idControlledObject"] == "object3")
    trial["C_O4"] = (trial["idControlledObject"] == "object4")

    trial = trial[list(YOKED_TRAINING_COLS) + ["idControlledObject"]].copy()

    true_mass = response["trueMass"]
    trial["A"] = (true_mass == "A")
    trial["B"] = (true_mass == "B")
    trial["same"] = (true_mass == "same")

    true_relationship = response["trueRelationship"]
    trial["attract"] = (true_relationship == "attract")
    trial["none"] = (true_relationship == "none")
    trial["repel"] = (true_relationship == "repel")
        
    trial.to_hdf(path_or_buf=hdf_path, key="trial_"+str(trial_i))

print(velocities_not_recorded / 11, "participants for which velocities weren't recorded")
skipped=0

100%|██████████| 216/216 [00:04<00:00, 43.88it/s]

4.909090909090909 participants for which velocities weren't recorded
